# COMP9318 Lab2

## Instructions
1. This note book contains instructions for **COMP9318-lab2**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures return by corresponding functions.

* You need to submit the code for **lab2** via following link: http://kg.cse.unsw.edu.au:8318/lab2/ .

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* If you choose to skip a question, leave the corresponding function body as it is (i.e., keep the `pass` line), otherwise it may affect your mark for other questions.

* You are allowed to add other functions and/or import additional modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **11:59:59 PM on 4th April, 2018**. We will **not** accept any late submissions.

# Question 1: Optimized BUC algorithm (50 points)

You need to implement the full `buc_rec_optimized` algorithm with the single-tuple optimization (as described below). Given an input dataframe:

 A | B | M 
---|---|---
 1 | 2 | 100
 2 | 1 | 20

Invoking  `buc_rec_optimized` on this data will result in following dataframe: 


 A | B | M
---|---|---
 1 | 2 | 100
 1 |ALL| 100
 2 | 1 | 20 
 2 |ALL| 20
ALL| 1 | 20
ALL| 2 | 100
ALL|ALL| 120

In the file `submission.py`, we have pre-defined the `buc_rec_optimized` function and its helper functions. 

## Input and output

Both input and output are dataframes.

The input dataframe (i.e., the base cuboid) is directly generated from the input file. Given the dimensionality of the base cuboid is $d$, each row is like:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 

The output dataframe contains $n$ rows, each for a non-empty cell in the compute data cube derived from the input base cuboid. Each row is formatted like input:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 


## The single-tuple optimization

Consider the naive way of recursive implementation of the BUC algorithm, you will notice that it uses several recursive calls to compute all the derived results from an input that consists of only one tuple. This is certainly a waste of computation. 

For example, if we are asked to compute the cube given the following input

 B | C | M 
---|---|---
 1 | 2 | 100

We can immmediately output the following, **without** using any recursive calls. 

<pre>
1    2    100
*    2    100
1    *    100
*    *    100
</pre>



** Note: For lab-2, you are allowed to use only two libraries, i.e., pandas, and numpy.** 

In [ ]:
import pandas as pd
import numpy as np

In [5]:
import pandas as pd
import numpy as np
##============================================================
# Data file format: 
# * tab-delimited input file
# * 1st line: dimension names and the last dimension is assumed to be the measure
# * rest of the lines: data values.
#DEBUG = True
DEBUG = False
def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)
      
# helper functions
def project_data(df, d):
    # Return only the d-th column of INPUT
    return df.iloc[:, d]

def select_data(df, d, val):
    # SELECT * FROM INPUT WHERE input.d = val
    col_name = df.columns[d]
    return df[df[col_name] == val]

def remove_first_dim(df):
    # Remove the first dim of the input
    return df.iloc[:, 1:]

def slice_data_dim0(df, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(df, 0, v)
    return remove_first_dim(df_temp)
def dump_input2(input):
    if DEBUG: 
        print("\n.. BUC_rec invoked on:")
        print(input)
        print("......................\n")
def copy(src_list):
    return [i for i in src_list]
def cart_product(row, cols):
    dims = row[:-1][1-cols:]
    dim = len(dims)
    result = []
    for i in range(2 ** dim):
        new_row = list(row)
        for j in range(dim):
            gap = 2 ** j
            if i % (2 * gap) > gap -1: 
                new_row[len(new_row) - 2 -j] = 'ALL'
        result.append(new_row)
    return result

def buc_rec_0(input, result, prev = []):
    # Note that input is a DataFrame
    #dump_input2(input)
    dims = input.shape[1]
    rows = input.shape[0]
    new_prev = copy(prev)
    if rows ==1 and dims > 1:
        #print("haha")
        new_prev += input.iloc[0,].tolist()
        #for i in range(len(rows)-1):
        #    rows[i] = [str(rows[i]), '*']
        new_rows = cart_product(tuple(new_prev), dims)
        result += new_rows
        #print("sssss",result)
        return result
    if dims == 1:
        # only the measure dim
        input_sum = sum( project_data(input, 0) )
        new_prev.append(input_sum)
        result.append(new_prev)
        return result
        #for i in prev:
        #    print('%-5s'%str(i), end='')
        #output(input_sum)
    else:
        # the general case
        dim0_vals = set(project_data(input, 0).values)
        for dim0_v in dim0_vals:
            new_prev.append(dim0_v)
            sub_data = slice_data_dim0(input, dim0_v)
            result = buc_rec_0(sub_data, result, new_prev)
            new_prev = copy(prev)
        ## for R_{ALL}
        sub_data = remove_first_dim(input)
        new_prev.append('ALL')
        result = buc_rec_0(sub_data, result, new_prev)
        #print("aaa",result)
    return result
def buc_rec_optimized(df):# do not change the heading of the function
    result = []
    result = buc_rec_0(df,result)
    result = pd.DataFrame(result, columns = df.columns.values)
    return result
     

#data = read_data('./asset/a_.txt')
#rows = buc_rec_optimized(data)
#rows
## You can test your implementation using the following code...

import submission as submission
input_data = read_data('./asset/sale.txt')
output = submission.buc_rec_optimized(input_data)
output

,Location,Time,Item,Quantity
0,1,2005,1,1400
1,1,2005,ALL,1400
2,1,2006,1,1500
3,1,2006,2,500
4,1,2006,ALL,2000
5,1,ALL,1,2900
6,1,ALL,2,500
7,1,ALL,ALL,3400
8,2,2005,3,1700
9,2,2005,ALL,1700


# Question 2: Optimal binning algorithm using dynamic programming (50 points)

You need to implement the optimal binning algorithm using the dynamic programming algorithm we discussed in the lecture. You are allowed to use $O(n^2)$ space. 

## Input

The input contains data (in a list) and the number of bins (an integer).

## Output

You are required to output the binning result and the matrix computed by the algorithm.

The matrix entries record optimal binning cost for a suffix of the input array using a certain number of bins. You should assign -1 to all the invalid solutions.

In [ ]:
x = [3, 1, 18, 11, 13, 17]
num_bins = 4

In [4]:
import pandas as pd
import numpy as np

def partition(List):
    for i in range(1, len(List)):
        for j in partition(List[i:]):
            yield [List[:i]] + j
    yield [List]

def bin_cost(partition,cost_dict):
    cost = 0
    for part in partition:
        if part in cost_dict:
            cost += cost_dict[part]
        else:
            new_cost = sse(part)  
            cost_dict[part] = new_cost
            cost += new_cost
    return cost,cost_dict

def sse(arr):
    if len(arr) == 0: # deal with arr == []
        return 0.0

    avg = np.average(arr)
    val = sum([(x-avg)*(x-avg) for x in arr] )

    return val

def v_opt_dp(x, num_bins):# do not change the heading of the function
    x= tuple(x)
    sfxLen = len(x)
    results = [[-1] * sfxLen] * num_bins
    costs = [[-1]* sfxLen] * num_bins 
    cost_dict = dict()

    # Loop for suffix then loop for bins
    for bins in range(1,num_bins + 1):
        for i in reversed(range(sfxLen)):
            suffix = x[i : ]
            print("bins, suffix, len",[bins,suffix,len(suffix)])
            # Check if quesiton is valid
            if bins > len(suffix):
                continue
            # Don't call function for one bin
            elif bins == 1:
                cost = sse(suffix)
                cost_dict[suffix] = cost
                optimal = suffix
            else:

                # Build partitions
                partitions = [part for part in partition(suffix) if len(part) == bins]
                partition_costs = []

                # Calculate costs
                for part in partitions:
                    cost,cost_dict = bin_cost(part, cost_dict)
                    partition_costs.append(cost)
                
                # Get minimums
                cost = min(partition_costs)
                optimal = partitions[partition_costs.index(min(partition_costs))]

            # Insert cost and solution
            costs[bins-1][i] = cost
            results[bins-1][i] = optimal
    print("bins",bins)
    return costs,results[bins-1][0]    
x = [3, 1, 18, 11, 13, 17]
num_bins = 4
matrix, bins = v_opt_dp(x, num_bins)
print("Bins = {}".format(bins))
print("Matrix =")
for row in matrix:
    print(row)

bins, suffix, len [1, (17,), 1]
bins, suffix, len [1, (13, 17), 2]
bins, suffix, len [1, (11, 13, 17), 3]
bins, suffix, len [1, (18, 11, 13, 17), 4]
bins, suffix, len [1, (1, 18, 11, 13, 17), 5]
bins, suffix, len [1, (3, 1, 18, 11, 13, 17), 6]
bins, suffix, len [2, (17,), 1]
bins, suffix, len [2, (13, 17), 2]
bins, suffix, len [2, (11, 13, 17), 3]
bins, suffix, len [2, (18, 11, 13, 17), 4]
bins, suffix, len [2, (1, 18, 11, 13, 17), 5]
bins, suffix, len [2, (3, 1, 18, 11, 13, 17), 6]
bins, suffix, len [3, (17,), 1]
bins, suffix, len [3, (13, 17), 2]
bins, suffix, len [3, (11, 13, 17), 3]
bins, suffix, len [3, (18, 11, 13, 17), 4]
bins, suffix, len [3, (1, 18, 11, 13, 17), 5]
bins, suffix, len [3, (3, 1, 18, 11, 13, 17), 6]
bins, suffix, len [4, (17,), 1]
bins, suffix, len [4, (13, 17), 2]
bins, suffix, len [4, (11, 13, 17), 3]
bins, suffix, len [4, (18, 11, 13, 17), 4]
bins, suffix, len [4, (1, 18, 11, 13, 17), 5]
bins, suffix, len [4, (3, 1, 18, 11, 13, 17), 6]
bins 4
Bins = [(3, 1), (

In [ ]:
## You can test your implementation using the following code...

import submission as submission
matrix, bins = submission.v_opt_dp(x, num_bins)
print("Bins = {}".format(bins))
print("Matrix =")
for row in matrix:
    print(row)

In [61]:
import pandas as pd
import numpy as np

def partition1(data):
    print("sasas",data)
    for i in range(1, len(data)):
        for j in partition(data[i:]):
            yield [data[:i]] + j
    yield [data]
def partition(List):
    for i in range(1, len(List)):
        for j in partition(List[i:]):
            yield [List[:i]] + j
    yield [List]
    
def sse(arr):
    if len(arr) == 0:
        return 0.0
    avg = np.average(arr)
    val = sum( [(x-avg)*(x-avg) for x in arr] )
    return val

def bin_cost(partition,costs):
    cost_sum = 0
    for part in partition:
        if part not in costs:
            costs[part] = sse(part)
        cost_sum += costs[part]
    #print(cost_sum, costs)
    return cost_sum, costs
        
def v_opt_dp(x, num_bins):# do not change the heading of the function
    x = tuple(x)
    sfxLen = len(x)
    M_Result = [-1 for x in range(sfxLen)]
    M_Cost = [[-1 for x in range(sfxLen)] for y in range(num_bins)]
    costs = {}
    for bins in range(1, num_bins+1):
        for i in reversed(range(sfxLen)):
            suffix = x[i:]
            prefix = x[:i]
            if len(prefix) + bins >= num_bins:
                if bins > len(suffix) :
                    continue
                elif bins == 1:
                    min_cost = sse(suffix)
                    costs[suffix] = min_cost
                    opt = suffix
                else:
                    partitions = [part for part in partition(suffix) if len(part) == bins]
                    #print("partitions",partitions)
                    par_costs = []
                    for part in partitions:
                        cost, costs = bin_cost(part, costs)
                        par_costs.append(cost)
                    #if len(par_costs) != 0:
                    min_cost = min(par_costs)
                    opt = partitions[par_costs.index(min_cost)]
                M_Cost[bins - 1][i] = min_cost
                #opt = list(opt)
                #print(opt)
                M_Result[i] = opt
    for i in range(len(M_Result[0])):
        M_Result[0][i] = list(M_Result[0][i])
    return M_Cost, M_Result[0]
x = [3,1,18,11,13,17]
num_bins = 4
matrix, bins = v_opt_dp(x, num_bins)
print("Bins = {}".format(bins))
print("Matrix =")
for row in matrix:
    print(row)

Bins = [[3, 1], [18], [11, 13], [17]]
Matrix =
[-1, -1, -1, 18.666666666666664, 8.0, 0.0]
[-1, -1, 18.666666666666664, 2.0, 0.0, -1]
[-1, 18.666666666666664, 2.0, 0.0, -1, -1]
[4.0, 2.0, 0.0, -1, -1, -1]


In [12]:
import numpy as np
def partition(lst):
    for i in range(1, len(lst)):
        for r in partition(lst[i:]):
            yield [lst[:i]] + r
    yield [lst]

def bin_cost(partition,cost_dict):
    cost = 0
    for part in partition:
        if part in cost_dict:
            cost += cost_dict[part]
        else:
            new_cost = sse(part)  
            cost_dict[part] = new_cost
            cost += new_cost
    return cost,cost_dict

def print_list_of_list(lol):
    for l in lol:
        print(l)


def v_opt_dp(x, num_bins):# do not change the heading of the function
    x= tuple(x)
    suffix_len = len(x)
    result_matrix = [[-1 for x in range(suffix_len)] for y in range(num_bins)]
    cost_matrix = [[-1 for x in range(suffix_len)] for y in range(num_bins)] 
    cost_dict = {}

    # Loop for suffix then loop for bins
    for bins in range(1,num_bins + 1):
        for i in reversed(range(suffix_len)):
            prefix = x[0 : i ]
            suffix = x[i  : ]

            # Check if quesiton is valid
            if bins > len(suffix):
                continue
            # Don't call function for one bin
            elif bins == 1:
                cost = sse(suffix)
                cost_dict[suffix] = cost
                optimal = suffix
            else:

                # Build partitions
                partitions = [part for part in partition(suffix) if len(part) == bins]
                partition_costs = []

                # Calculate costs
                for part in partitions:
                    cost,cost_dict = bin_cost(part, cost_dict)
                    partition_costs.append(cost)
                
                # Get minimums
                cost = min(partition_costs)
                optimal = partitions[partition_costs.index(min(partition_costs))]

            # Insert cost and solution
            cost_matrix[bins-1][i] = cost
            result_matrix[bins-1][i] = optimal
            print("optimal",optimal)

    return cost_matrix,result_matrix[bins-1][0]


def sse(arr):
    if len(arr) == 0: # deal with arr == []
        return 0.0

    avg = np.average(arr)
    val = sum([(x-avg)*(x-avg) for x in arr] )

    return val

def calc_depth(b):
    return 5 - b
x = [3,1,18,11,13,17]
num_bins = 4
matrix, bins = v_opt_dp(x, num_bins)
print("Bins = {}".format(bins))
print("Matrix =")
for row in matrix:
    print(row)

optimal (17,)
optimal (13, 17)
optimal (11, 13, 17)
optimal (18, 11, 13, 17)
optimal (1, 18, 11, 13, 17)
optimal (3, 1, 18, 11, 13, 17)
optimal [(13,), (17,)]
optimal [(11, 13), (17,)]
optimal [(18,), (11, 13, 17)]
optimal [(1,), (18, 11, 13, 17)]
optimal [(3, 1), (18, 11, 13, 17)]
optimal [(11,), (13,), (17,)]
optimal [(18,), (11, 13), (17,)]
optimal [(1,), (18,), (11, 13, 17)]
optimal [(3, 1), (18,), (11, 13, 17)]
optimal [(18,), (11,), (13,), (17,)]
optimal [(1,), (18,), (11, 13), (17,)]
optimal [(3, 1), (18,), (11, 13), (17,)]
Bins = [(3, 1), (18,), (11, 13), (17,)]
Matrix =
[251.5, 184.0, 32.75, 18.666666666666664, 8.0, 0.0]
[34.75, 32.75, 18.666666666666664, 2.0, 0.0, -1]
[20.666666666666664, 18.666666666666664, 2.0, 0.0, -1, -1]
[4.0, 2.0, 0.0, -1, -1, -1]


In [59]:
a = (17,13,11)
list(a)

[17, 13, 11]